In [15]:
from word2vec_functions import *
import h2o
h2o.init()
import numpy as np
import pandas as pd
from h2o.estimators.word2vec import H2OWord2vecEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from datetime import datetime

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,12 mins 09 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,26 days
H2O_cluster_name:,H2O_from_python_csridlen_2phdue
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.765 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [19]:
# Initializing word2vec model
airbnb_path = "../data/airbnb_listings_2021.csv"
airbnb_names = h2o.import_file(airbnb_path, destination_frame = "airbnbnames",
                             header = 1)
# Create word vectors
words = tokenize(airbnb_names["name"])
w2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 10)
w2v_model.train(training_frame = words)
airbnb_names_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")


print(airbnb_names_vecs.dim) # 36923 x 100
print(airbnb_names.dim) # 36923 x 18

# Merge with original dataset to add as a feature


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


NameError: name 'word_2vec' is not defined

[36923, 100]
[36923, 18]
